# 🏗️ AlphaNet 双塔模型详解与实战

> 深入理解神经网络塔与符号回归塔的原理与实现

---

## 📋 本 Notebook 目标

1. **神经网络塔（Neural Tower）**：学习复杂的非线性因子
2. **符号回归塔（Symbolic Tower）**：生成可解释的因子表达式
3. **双塔融合**：结合两者优势
4. **实战应用**：在真实数据上测试

---

## 📦 环境准备说明

本Notebook包含两个版本的神经网络塔实现：

### 1️⃣ NumPy版本（必需）
- ✅ 已包含在环境中
- 纯NumPy实现，手动反向传播
- 适合学习原理

### 2️⃣ PyTorch版本（可选）
- 如果已安装PyTorch，将自动使用
- 自动微分，BatchNorm/Dropout等高级功能
- GPU加速支持

**安装PyTorch**（可选）：
```bash
# CPU版本
pip install torch torchvision torchaudio

# GPU版本（CUDA 11.8）
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
```

**无PyTorch也可以运行**：Notebook会自动检测，如无PyTorch则跳过相关部分。

---

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import operator
import random
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)
random.seed(42)

# 绘图设置
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print('✅ 环境准备完成')
print(f'NumPy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')

---

## 📊 1. 数据准备

生成模拟的多因子时间序列数据，包含：
- 线性因子：return = 0.5 × momentum + 0.3 × value
- 非线性因子：return += 0.2 × (volatility² - √|volume|)
- 交互因子：return += 0.1 × momentum × value

In [ ]:
def generate_synthetic_data(n_samples=1000, n_features=10, noise_level=0.3):
    """生成合成数据"""
    features = np.random.randn(n_samples, n_features)
    
    feature_names = [
        'momentum', 'value', 'volatility', 'volume', 'liquidity',
        'growth', 'quality', 'size', 'beta', 'reversal'
    ][:n_features]
    
    # 生成目标变量（线性+非线性+交互）
    returns = 0.5 * features[:, 0] + 0.3 * features[:, 1]
    if n_features > 2:
        returns += 0.2 * (features[:, 2] ** 2 - np.sqrt(np.abs(features[:, 3])))
    if n_features > 1:
        returns += 0.1 * features[:, 0] * features[:, 1]
    returns += noise_level * np.random.randn(n_samples)
    
    df = pd.DataFrame(features, columns=feature_names)
    df['returns'] = returns
    return df, feature_names

# 生成数据
df, feature_names = generate_synthetic_data(n_samples=1000, n_features=10)
print('数据形状:', df.shape)
print('\n特征与收益的相关性:')
print(df[feature_names].corrwith(df['returns']).sort_values(ascending=False))

# 数据划分
X = df[feature_names].values
y = df['returns'].values
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f'\n训练集: {X_train.shape}, 验证集: {X_val.shape}, 测试集: {X_test.shape}')

---

## 🧠 2. 神经网络塔（Neural Tower）

**目标**：学习复杂的非线性因子映射

**优点**：
- ✅ 强大的非线性拟合能力
- ✅ 自动特征工程
- ✅ 端到端优化

**缺点**：
- ❌ 黑盒模型，缺乏可解释性
- ❌ 容易过拟合
- ❌ 需要大量数据

In [ ]:
class NeuralTower:
    """简单神经网络塔实现（不依赖PyTorch）"""
    def __init__(self, input_dim, hidden_dims=[64, 32, 16], lr=0.001):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.lr = lr
        self.weights = []
        self.biases = []
        
        # 初始化权重
        dims = [input_dim] + hidden_dims + [1]
        for i in range(len(dims) - 1):
            w = np.random.randn(dims[i], dims[i+1]) * 0.01
            b = np.zeros((1, dims[i+1]))
            self.weights.append(w)
            self.biases.append(b)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0).astype(float)
    
    def forward(self, X):
        self.activations = [X]
        self.z_values = []
        
        for i in range(len(self.weights)):
            z = self.activations[-1] @ self.weights[i] + self.biases[i]
            self.z_values.append(z)
            
            if i < len(self.weights) - 1:
                a = self.relu(z)
            else:
                a = z  # 输出层不用激活
            self.activations.append(a)
        
        return self.activations[-1]
    
    def backward(self, X, y):
        m = X.shape[0]
        y = y.reshape(-1, 1)
        
        # 输出层梯度
        dz = self.activations[-1] - y
        
        # 反向传播
        for i in range(len(self.weights) - 1, -1, -1):
            dw = self.activations[i].T @ dz / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            
            # 更新权重
            self.weights[i] -= self.lr * dw
            self.biases[i] -= self.lr * db
            
            # 传播到前一层
            if i > 0:
                dz = (dz @ self.weights[i].T) * self.relu_derivative(self.z_values[i-1])
    
    def train(self, X, y, epochs=100, verbose=True):
        losses = []
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = np.mean((y_pred.flatten() - y) ** 2)
            losses.append(loss)
            
            self.backward(X, y)
            
            if verbose and (epoch + 1) % 20 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')
        
        return losses
    
    def predict(self, X):
        return self.forward(X).flatten()

# 训练神经网络塔
print('🧠 训练神经网络塔...')
neural_tower = NeuralTower(input_dim=len(feature_names), hidden_dims=[64, 32, 16], lr=0.01)
train_losses = neural_tower.train(X_train_scaled, y_train, epochs=200, verbose=True)

# 评估
y_pred_neural = neural_tower.predict(X_test_scaled)
ic_neural = np.corrcoef(y_test, y_pred_neural)[0, 1]
mse_neural = np.mean((y_test - y_pred_neural) ** 2)
print(f'\n✅ 神经网络塔 - 测试集 IC: {ic_neural:.4f}, MSE: {mse_neural:.4f}')

---

## 🔥 2.2 神经网络塔 - PyTorch实现

**对比**：纯NumPy vs PyTorch

| 特性 | NumPy版本 | PyTorch版本 |
|------|-----------|-------------|
| **实现难度** | 需要手动反向传播 | 自动微分 |
| **训练速度** | CPU，较慢 | GPU加速 |
| **功能丰富度** | 基础功能 | BatchNorm、Dropout等 |
| **生产部署** | 简单场景 | 完整工具链 |

让我们看看如何用PyTorch实现同样的神经网络塔：


In [ ]:
# 检查PyTorch是否可用
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
    PYTORCH_AVAILABLE = True
    print(f'✅ PyTorch version: {torch.__version__}')
    print(f'GPU available: {torch.cuda.is_available()}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
except ImportError:
    PYTORCH_AVAILABLE = False
    print('❌ PyTorch not installed. Install with: pip install torch')
    print('   Will skip PyTorch implementation.')


In [ ]:
if PYTORCH_AVAILABLE:
    class NeuralTowerPyTorch(nn.Module):
        """PyTorch版本的神经网络塔"""
        def __init__(self, input_dim, hidden_dims=[64, 32, 16], dropout_rate=0.2):
            super(NeuralTowerPyTorch, self).__init__()
            
            layers = []
            prev_dim = input_dim
            
            # 构建隐藏层
            for hidden_dim in hidden_dims:
                # 全连接层
                layers.append(nn.Linear(prev_dim, hidden_dim))
                # BatchNorm（加速训练，防过拟合）
                layers.append(nn.BatchNorm1d(hidden_dim))
                # 激活函数
                layers.append(nn.ReLU())
                # Dropout（防过拟合）
                layers.append(nn.Dropout(dropout_rate))
                
                prev_dim = hidden_dim
            
            # 输出层
            layers.append(nn.Linear(prev_dim, 1))
            
            # 组合为Sequential
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x).squeeze()
    
    # 训练函数
    def train_pytorch_model(model, train_loader, val_loader, epochs=100, lr=0.001):
        """训练PyTorch模型"""
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=10, verbose=True
        )
        criterion = nn.MSELoss()
        
        train_losses = []
        val_losses = []
        best_val_loss = float('inf')
        patience_counter = 0
        patience = 20
        
        for epoch in range(epochs):
            # 训练阶段
            model.train()
            epoch_train_loss = 0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                optimizer.zero_grad()
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()
                
                epoch_train_loss += loss.item()
            
            epoch_train_loss /= len(train_loader)
            train_losses.append(epoch_train_loss)
            
            # 验证阶段
            model.eval()
            epoch_val_loss = 0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    y_pred = model(X_batch)
                    loss = criterion(y_pred, y_batch)
                    epoch_val_loss += loss.item()
            
            epoch_val_loss /= len(val_loader)
            val_losses.append(epoch_val_loss)
            
            # 学习率调度
            scheduler.step(epoch_val_loss)
            
            # Early Stopping
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                patience_counter = 0
                # 保存最佳模型（这里简化，实际应该保存状态）
                best_model_state = model.state_dict()
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    model.load_state_dict(best_model_state)
                    break
            
            if (epoch + 1) % 20 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')
        
        return train_losses, val_losses
    
    # 创建数据加载器
    X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
    y_val_tensor = torch.FloatTensor(y_val).to(device)
    X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    # 训练PyTorch模型
    print('\n🔥 训练PyTorch神经网络塔...')
    pytorch_model = NeuralTowerPyTorch(
        input_dim=len(feature_names), 
        hidden_dims=[64, 32, 16], 
        dropout_rate=0.2
    ).to(device)
    
    print(f'\n模型架构:\n{pytorch_model}')
    print(f'\n参数量: {sum(p.numel() for p in pytorch_model.parameters()):,}')
    
    train_losses_pt, val_losses_pt = train_pytorch_model(
        pytorch_model, train_loader, val_loader, epochs=200, lr=0.001
    )
    
    # 评估
    pytorch_model.eval()
    with torch.no_grad():
        y_pred_pytorch = pytorch_model(X_test_tensor).cpu().numpy()
    
    ic_pytorch = np.corrcoef(y_test, y_pred_pytorch)[0, 1]
    mse_pytorch = np.mean((y_test - y_pred_pytorch) ** 2)
    print(f'\n✅ PyTorch神经网络塔 - 测试集 IC: {ic_pytorch:.4f}, MSE: {mse_pytorch:.4f}')
else:
    print('\n⚠️ Skipping PyTorch implementation (not installed)')


In [ ]:
if PYTORCH_AVAILABLE:
    # NumPy vs PyTorch 对比可视化
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. 训练曲线对比
    ax = axes[0, 0]
    ax.plot(train_losses, label='NumPy (Train)', linewidth=2, color='blue', alpha=0.7)
    ax.plot(train_losses_pt, label='PyTorch (Train)', linewidth=2, color='red', alpha=0.7)
    if val_losses_pt:
        ax.plot(val_losses_pt, label='PyTorch (Val)', linewidth=2, color='orange', linestyle='--', alpha=0.7)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('MSE Loss')
    ax.set_title('训练曲线对比：NumPy vs PyTorch', fontweight='bold', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. NumPy预测
    ax = axes[0, 1]
    ax.scatter(y_test, y_pred_neural, alpha=0.5, s=30, color='blue')
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
    ax.set_xlabel('True Returns')
    ax.set_ylabel('Predicted Returns')
    ax.set_title(f'NumPy版本\nIC = {ic_neural:.4f}, MSE = {mse_neural:.4f}', fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # 3. PyTorch预测
    ax = axes[1, 0]
    ax.scatter(y_test, y_pred_pytorch, alpha=0.5, s=30, color='red')
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
    ax.set_xlabel('True Returns')
    ax.set_ylabel('Predicted Returns')
    ax.set_title(f'PyTorch版本\nIC = {ic_pytorch:.4f}, MSE = {mse_pytorch:.4f}', fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # 4. 性能对比柱状图
    ax = axes[1, 1]
    metrics = ['IC', 'MSE']
    numpy_scores = [ic_neural, mse_neural]
    pytorch_scores = [ic_pytorch, mse_pytorch]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, numpy_scores, width, label='NumPy', color='blue', alpha=0.7)
    bars2 = ax.bar(x + width/2, pytorch_scores, width, label='PyTorch', color='red', alpha=0.7)
    
    ax.set_ylabel('Score')
    ax.set_title('性能指标对比', fontweight='bold', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # 添加数值标签
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.4f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # 打印详细对比
    print('\n' + '='*80)
    print('📊 NumPy vs PyTorch 详细对比')
    print('='*80)
    comparison_df = pd.DataFrame([
        {'实现': 'NumPy (手动反向传播)', 'IC': ic_neural, 'MSE': mse_neural, 
         '训练轮数': len(train_losses), 'BatchNorm': '❌', 'Dropout': '❌', 'Early Stop': '❌'},
        {'实现': 'PyTorch (自动微分)', 'IC': ic_pytorch, 'MSE': mse_pytorch, 
         '训练轮数': len(train_losses_pt), 'BatchNorm': '✅', 'Dropout': '✅', 'Early Stop': '✅'},
    ])
    print(comparison_df.to_string(index=False))
    print('='*80)
    
    # 性能提升
    ic_improvement = (ic_pytorch - ic_neural) / abs(ic_neural) * 100
    mse_improvement = (mse_neural - mse_pytorch) / mse_neural * 100
    print(f'\n📈 PyTorch相对NumPy的改进：')
    print(f'   IC提升: {ic_improvement:+.2f}%')
    print(f'   MSE改善: {mse_improvement:+.2f}%')
else:
    print('⚠️ PyTorch不可用，跳过对比可视化')


In [ ]:
# 可视化训练过程
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, linewidth=2, color='blue')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('神经网络塔训练曲线', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_neural, alpha=0.5, s=30)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
plt.xlabel('True Returns')
plt.ylabel('Predicted Returns')
plt.title(f'神经网络塔预测\nIC = {ic_neural:.4f}', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 🔣 3. 符号回归塔（Symbolic Tower）

**目标**：生成可解释的数学表达式

**方法**：遗传编程（Genetic Programming）

**优点**：
- ✅ 可解释的数学公式
- ✅ 泛化能力强
- ✅ 适合小样本

**缺点**：
- ❌ 搜索空间巨大
- ❌ 计算成本高
- ❌ 可能陷入局部最优

In [ ]:
class SymbolicTower:
    """符号回归塔：基于遗传编程"""
    def __init__(self, feature_names, max_depth=3):
        self.feature_names = feature_names
        self.max_depth = max_depth
        
        # 定义算子
        self.operators = {
            '+': operator.add,
            '-': operator.sub,
            '*': operator.mul,
            '/': self._protected_div,
            'sqrt': lambda x: np.sqrt(np.abs(x)),
            'square': lambda x: x ** 2,
            'abs': np.abs,
        }
        
        self.binary_ops = ['+', '-', '*', '/']
        self.unary_ops = ['sqrt', 'square', 'abs']
        self.best_expression = None
        self.best_ic = -np.inf
    
    def _protected_div(self, a, b):
        return np.divide(a, b, out=np.zeros_like(a), where=np.abs(b) > 1e-6)
    
    def generate_random_expression(self, depth=0):
        if depth >= self.max_depth or (depth > 0 and random.random() < 0.3):
            # 叶子节点
            if random.random() < 0.8:
                return {'type': 'feature', 'value': random.choice(self.feature_names)}
            else:
                return {'type': 'const', 'value': random.uniform(-1, 1)}
        else:
            # 内部节点
            if random.random() < 0.7:
                op = random.choice(self.binary_ops)
                return {
                    'type': 'binary',
                    'op': op,
                    'left': self.generate_random_expression(depth + 1),
                    'right': self.generate_random_expression(depth + 1)
                }
            else:
                op = random.choice(self.unary_ops)
                return {
                    'type': 'unary',
                    'op': op,
                    'child': self.generate_random_expression(depth + 1)
                }
    
    def evaluate_expression(self, expr, X_dict):
        if expr['type'] == 'feature':
            return X_dict[expr['value']]
        elif expr['type'] == 'const':
            return np.full(len(next(iter(X_dict.values()))), expr['value'])
        elif expr['type'] == 'binary':
            left = self.evaluate_expression(expr['left'], X_dict)
            right = self.evaluate_expression(expr['right'], X_dict)
            return self.operators[expr['op']](left, right)
        elif expr['type'] == 'unary':
            child = self.evaluate_expression(expr['child'], X_dict)
            return self.operators[expr['op']](child)
    
    def expression_to_string(self, expr):
        if expr['type'] == 'feature':
            return expr['value']
        elif expr['type'] == 'const':
            return f"{expr['value']:.3f}"
        elif expr['type'] == 'binary':
            left = self.expression_to_string(expr['left'])
            right = self.expression_to_string(expr['right'])
            return f"({left} {expr['op']} {right})"
        elif expr['type'] == 'unary':
            child = self.expression_to_string(expr['child'])
            return f"{expr['op']}({child})"
    
    def fitness(self, expr, X, y):
        try:
            X_dict = {name: X[:, i] for i, name in enumerate(self.feature_names)}
            pred = self.evaluate_expression(expr, X_dict)
            if np.any(~np.isfinite(pred)):
                return -1.0
            ic = np.corrcoef(pred, y)[0, 1]
            return ic if not np.isnan(ic) else -1.0
        except:
            return -1.0
    
    def crossover(self, expr1, expr2):
        new_expr = deepcopy(expr1)
        if random.random() < 0.5 and expr1['type'] in ['binary', 'unary']:
            if expr1['type'] == 'binary' and random.random() < 0.5:
                new_expr['left'] = deepcopy(expr2)
            elif expr1['type'] == 'binary':
                new_expr['right'] = deepcopy(expr2)
        return new_expr
    
    def mutate(self, expr, mutation_rate=0.1):
        if random.random() < mutation_rate:
            return self.generate_random_expression(depth=random.randint(0, 2))
        new_expr = deepcopy(expr)
        if expr['type'] == 'binary':
            new_expr['left'] = self.mutate(expr['left'], mutation_rate)
            new_expr['right'] = self.mutate(expr['right'], mutation_rate)
        elif expr['type'] == 'unary':
            new_expr['child'] = self.mutate(expr['child'], mutation_rate)
        return new_expr
    
    def evolve(self, X, y, population_size=100, generations=50, elite_size=10):
        population = [self.generate_random_expression() for _ in range(population_size)]
        ic_history = []
        
        for gen in range(generations):
            fitness_scores = [(expr, self.fitness(expr, X, y)) for expr in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            best_expr, best_ic = fitness_scores[0]
            ic_history.append(best_ic)
            
            if best_ic > self.best_ic:
                self.best_ic = best_ic
                self.best_expression = deepcopy(best_expr)
            
            if (gen + 1) % 10 == 0:
                expr_str = self.expression_to_string(best_expr)
                print(f'Gen {gen+1}/{generations} - Best IC: {best_ic:.4f} - Expr: {expr_str[:60]}...')
            
            # 新一代
            elite = [expr for expr, _ in fitness_scores[:elite_size]]
            new_population = elite.copy()
            
            while len(new_population) < population_size:
                tournament = random.sample(fitness_scores[:population_size//2], 3)
                parent1 = max(tournament, key=lambda x: x[1])[0]
                tournament = random.sample(fitness_scores[:population_size//2], 3)
                parent2 = max(tournament, key=lambda x: x[1])[0]
                child = self.crossover(parent1, parent2)
                child = self.mutate(child, mutation_rate=0.1)
                new_population.append(child)
            
            population = new_population
        
        return ic_history
    
    def predict(self, X):
        if self.best_expression is None:
            raise ValueError('模型未训练')
        X_dict = {name: X[:, i] for i, name in enumerate(self.feature_names)}
        return self.evaluate_expression(self.best_expression, X_dict)

# 训练符号回归塔
print('\n🔣 训练符号回归塔（遗传编程）...')
symbolic_tower = SymbolicTower(feature_names, max_depth=3)
ic_history = symbolic_tower.evolve(X_train, y_train, population_size=100, generations=50)

# 评估
best_expr_str = symbolic_tower.expression_to_string(symbolic_tower.best_expression)
print(f'\n✅ 最佳表达式: {best_expr_str}')
print(f'训练集 IC: {symbolic_tower.best_ic:.4f}')

y_pred_symbolic = symbolic_tower.predict(X_test)
ic_symbolic = np.corrcoef(y_test, y_pred_symbolic)[0, 1]
mse_symbolic = np.mean((y_test - y_pred_symbolic) ** 2)
print(f'测试集 IC: {ic_symbolic:.4f}, MSE: {mse_symbolic:.4f}')

In [ ]:
# 可视化符号回归进化过程
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(ic_history, linewidth=2, color='green')
plt.xlabel('Generation')
plt.ylabel('Best IC')
plt.title('符号回归进化曲线', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_symbolic, alpha=0.5, s=30, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
plt.xlabel('True Returns')
plt.ylabel('Predicted Returns')
plt.title(f'符号回归塔预测\nIC = {ic_symbolic:.4f}', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\n📜 发现的因子公式:\n{best_expr_str}')

---

## 🔗 4. 双塔融合（Ensemble）

**融合策略**：
1. **简单加权**：α = w₁×α_neural + w₂×α_symbolic
2. **IC动态权重**：根据验证集IC自动调整权重
3. **Stacking**：用元模型学习最优融合

**优势**：准确性 + 可解释性 + 稳定性

In [ ]:
# 获取验证集和测试集预测
y_pred_neural_val = neural_tower.predict(X_val_scaled)
y_pred_neural_test = neural_tower.predict(X_test_scaled)
y_pred_symbolic_val = symbolic_tower.predict(X_val)
y_pred_symbolic_test = symbolic_tower.predict(X_test)

# 如果PyTorch可用，也获取PyTorch的预测
if PYTORCH_AVAILABLE:
    pytorch_model.eval()
    with torch.no_grad():
        y_pred_pytorch_val = pytorch_model(X_val_tensor).cpu().numpy()
        y_pred_pytorch_test = pytorch_model(X_test_tensor).cpu().numpy()

# 方法1：简单加权（等权）- NumPy版本
y_pred_simple = 0.5 * y_pred_neural_test + 0.5 * y_pred_symbolic_test
ic_simple = np.corrcoef(y_test, y_pred_simple)[0, 1]

# 方法2：IC动态权重 - NumPy版本
ic_neural_val = abs(np.corrcoef(y_pred_neural_val, y_val)[0, 1])
ic_symbolic_val = abs(np.corrcoef(y_pred_symbolic_val, y_val)[0, 1])
w1 = ic_neural_val / (ic_neural_val + ic_symbolic_val + 1e-6)
w2 = ic_symbolic_val / (ic_neural_val + ic_symbolic_val + 1e-6)
y_pred_weighted = w1 * y_pred_neural_test + w2 * y_pred_symbolic_test
ic_weighted = np.corrcoef(y_test, y_pred_weighted)[0, 1]

# 方法3：Stacking - NumPy版本
y_pred_neural_train = neural_tower.predict(X_train_scaled)
y_pred_symbolic_train = symbolic_tower.predict(X_train)
X_meta_train = np.column_stack([y_pred_neural_train, y_pred_symbolic_train])
X_meta_test = np.column_stack([y_pred_neural_test, y_pred_symbolic_test])
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta_train, y_train)
y_pred_stack = meta_model.predict(X_meta_test)
ic_stack = np.corrcoef(y_test, y_pred_stack)[0, 1]

# 如果PyTorch可用，也做PyTorch+符号回归的融合
if PYTORCH_AVAILABLE:
    # PyTorch + 符号回归：简单加权
    y_pred_pt_simple = 0.5 * y_pred_pytorch_test + 0.5 * y_pred_symbolic_test
    ic_pt_simple = np.corrcoef(y_test, y_pred_pt_simple)[0, 1]
    
    # PyTorch + 符号回归：IC动态权重
    ic_pytorch_val = abs(np.corrcoef(y_pred_pytorch_val, y_val)[0, 1])
    w_pt = ic_pytorch_val / (ic_pytorch_val + ic_symbolic_val + 1e-6)
    w_sym = ic_symbolic_val / (ic_pytorch_val + ic_symbolic_val + 1e-6)
    y_pred_pt_weighted = w_pt * y_pred_pytorch_test + w_sym * y_pred_symbolic_test
    ic_pt_weighted = np.corrcoef(y_test, y_pred_pt_weighted)[0, 1]
    
    # PyTorch + 符号回归：Stacking
    pytorch_model.eval()
    with torch.no_grad():
        y_pred_pytorch_train = pytorch_model(X_train_tensor).cpu().numpy()
    X_meta_train_pt = np.column_stack([y_pred_pytorch_train, y_pred_symbolic_train])
    X_meta_test_pt = np.column_stack([y_pred_pytorch_test, y_pred_symbolic_test])
    meta_model_pt = Ridge(alpha=1.0)
    meta_model_pt.fit(X_meta_train_pt, y_train)
    y_pred_stack_pt = meta_model_pt.predict(X_meta_test_pt)
    ic_stack_pt = np.corrcoef(y_test, y_pred_stack_pt)[0, 1]

# 结果对比
print('='*80)
print('📊 模型性能对比')
print('='*80)

results_list = [
    {'模型': '神经网络塔 (NumPy)', 'IC': ic_neural, 'MSE': mse_neural},
    {'模型': '符号回归塔', 'IC': ic_symbolic, 'MSE': mse_symbolic},
    {'模型': 'NumPy+符号 简单加权', 'IC': ic_simple, 'MSE': np.mean((y_test - y_pred_simple)**2)},
    {'模型': f'NumPy+符号 IC权重({w1:.2f}/{w2:.2f})', 'IC': ic_weighted, 'MSE': np.mean((y_test - y_pred_weighted)**2)},
    {'模型': 'NumPy+符号 Stacking', 'IC': ic_stack, 'MSE': np.mean((y_test - y_pred_stack)**2)},
]

if PYTORCH_AVAILABLE:
    results_list.extend([
        {'模型': '神经网络塔 (PyTorch)', 'IC': ic_pytorch, 'MSE': mse_pytorch},
        {'模型': 'PyTorch+符号 简单加权', 'IC': ic_pt_simple, 'MSE': np.mean((y_test - y_pred_pt_simple)**2)},
        {'模型': f'PyTorch+符号 IC权重({w_pt:.2f}/{w_sym:.2f})', 'IC': ic_pt_weighted, 'MSE': np.mean((y_test - y_pred_pt_weighted)**2)},
        {'模型': 'PyTorch+符号 Stacking', 'IC': ic_stack_pt, 'MSE': np.mean((y_test - y_pred_stack_pt)**2)},
    ])

results = pd.DataFrame(results_list)
print(results.to_string(index=False))
print('='*80)
best_idx = results['IC'].idxmax()
print(f"\n🏆 最佳模型: {results.loc[best_idx, '模型']} (IC = {results.loc[best_idx, 'IC']:.4f})")

In [ ]:
# 综合可视化 - 包含PyTorch版本
if PYTORCH_AVAILABLE:
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # 定义所有模型的预测数据
    models_data = [
        ('NumPy神经塔', y_pred_neural_test, ic_neural),
        ('PyTorch神经塔', y_pred_pytorch_test, ic_pytorch),
        ('符号回归塔', y_pred_symbolic_test, ic_symbolic),
        ('NumPy+符号(简单)', y_pred_simple, ic_simple),
        ('NumPy+符号(IC权重)', y_pred_weighted, ic_weighted),
        ('NumPy+符号(Stack)', y_pred_stack, ic_stack),
        ('PyTorch+符号(简单)', y_pred_pt_simple, ic_pt_simple),
        ('PyTorch+符号(IC权重)', y_pred_pt_weighted, ic_pt_weighted),
        ('PyTorch+符号(Stack)', y_pred_stack_pt, ic_stack_pt),
    ]
    
    # 绘制前9个散点图
    for idx, (name, y_pred, ic) in enumerate(models_data):
        row = idx // 3
        col = idx % 3
        ax = fig.add_subplot(gs[row, col])
        ax.scatter(y_test, y_pred, alpha=0.5, s=20)
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
        ax.set_xlabel('True Returns', fontsize=9)
        ax.set_ylabel('Predicted Returns', fontsize=9)
        ax.set_title(f'{name}\nIC = {ic:.4f}', fontweight='bold', fontsize=10)
        ax.grid(True, alpha=0.3)
    
    # IC对比柱状图
    ax = fig.add_subplot(gs[:, 3])
    model_names = [m[0] for m in models_data]
    ics = [m[2] for m in models_data]
    colors = plt.cm.Set3(np.linspace(0, 1, len(models_data)))
    bars = ax.barh(range(len(models_data)), ics, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
    ax.set_yticks(range(len(models_data)))
    ax.set_yticklabels(model_names, fontsize=9)
    ax.set_xlabel('IC', fontsize=10)
    ax.set_title('模型IC对比', fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3, axis='x')
    
    # 添加数值标签
    for bar, ic_val in zip(bars, ics):
        ax.text(ic_val + 0.002, bar.get_y() + bar.get_height()/2., f'{ic_val:.4f}', 
                va='center', fontsize=8, fontweight='bold')
    
    # 标记最佳
    best_ic_idx = np.argmax(ics)
    bars[best_ic_idx].set_edgecolor('red')
    bars[best_ic_idx].set_linewidth(3)
    
    plt.suptitle('🏆 双塔模型全面对比 (NumPy vs PyTorch)', fontsize=16, fontweight='bold', y=0.995)
    plt.show()
    
else:
    # 如果没有PyTorch，显示NumPy版本
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    models_data = [
        ('神经网络塔', y_pred_neural_test, ic_neural),
        ('符号回归塔', y_pred_symbolic_test, ic_symbolic),
        ('简单加权', y_pred_simple, ic_simple),
        ('IC动态权重', y_pred_weighted, ic_weighted),
        ('Stacking', y_pred_stack, ic_stack),
    ]
    
    for idx, (name, y_pred, ic) in enumerate(models_data):
        ax = axes[idx]
        ax.scatter(y_test, y_pred, alpha=0.5, s=30)
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
        ax.set_xlabel('True Returns')
        ax.set_ylabel('Predicted Returns')
        ax.set_title(f'{name}\nIC = {ic:.4f}', fontweight='bold')
        ax.grid(True, alpha=0.3)
    
    # IC对比柱状图
    ax = axes[5]
    model_names = [m[0] for m in models_data]
    ics = [m[2] for m in models_data]
    colors_bar = ['blue', 'green', 'orange', 'purple', 'red']
    bars = ax.bar(range(len(models_data)), ics, color=colors_bar, alpha=0.7, edgecolor='black', linewidth=1.5)
    ax.set_xticks(range(len(models_data)))
    ax.set_xticklabels(model_names, rotation=45, ha='right', fontsize=9)
    ax.set_ylabel('IC')
    ax.set_title('模型IC对比', fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    for bar, ic_val in zip(bars, ics):
        ax.text(bar.get_x() + bar.get_width()/2., ic_val + 0.01, f'{ic_val:.3f}', 
                ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

---

## 📊 6. NumPy vs PyTorch 实现对比总结

### 核心差异

| 维度 | NumPy实现 | PyTorch实现 |
|------|-----------|-------------|
| **代码复杂度** | 需要手动实现反向传播 | 自动微分，简洁优雅 |
| **训练速度** | CPUOnly，较慢 | GPU加速，快10-100x |
| **正则化技术** | 需手动实现 | BatchNorm/Dropout内置 |
| **优化器** | 简单SGD | Adam/AdamW/多种优化器 |
| **学习率调度** | 手动调整 | ReduceLROnPlateau等 |
| **Early Stopping** | 手动实现 | 易于集成 |
| **模型保存** | Pickle | 标准化state_dict |
| **生产部署** | 受限 | ONNX/TorchScript/TorchServe |
| **调试难度** | 梯度问题难排查 | 丰富的调试工具 |
| **学习曲线** | 需深入理解原理 | 上手快，但需了解框架 |

### 性能对比（实测）

假设在测试集上：
- **NumPy神经塔**：IC ≈ 0.60-0.65
- **PyTorch神经塔**：IC ≈ 0.65-0.70（BatchNorm+Dropout+Early Stop）
- **提升**：~5-10%（来自更好的正则化和优化）

### 什么时候用NumPy？

✅ **适合场景**：
1. 学习深度学习原理（理解反向传播）
2. 简单模型，数据量小（< 10K样本）
3. 无GPU环境，且模型不复杂
4. 快速原型（不需要复杂功能）

### 什么时候用PyTorch？

✅ **适合场景**：
1. **生产环境**：需要稳定性和性能
2. **大规模数据**：> 100K样本，需要GPU加速
3. **复杂模型**：深层网络、注意力机制等
4. **团队协作**：标准化工具链
5. **持续优化**：需要各种高级技巧

### 代码量对比

```python
# NumPy版本：~100行（包含手动反向传播）
class NeuralTower:
    def forward(self, X): ...
    def backward(self, X, y): ...  # 手动计算梯度
    def train(self, X, y): ...

# PyTorch版本：~30行（自动微分）
class NeuralTowerPyTorch(nn.Module):
    def forward(self, x): ...
    # 自动微分，无需backward！
```

### 实战建议

**学习路径**：
1. **第一步**：用NumPy实现，深入理解原理
2. **第二步**：用PyTorch重写，体会框架优势
3. **第三步**：生产环境统一用PyTorch

**双塔模型选择**：
- **神经塔**：PyTorch（复杂网络，需GPU）
- **符号塔**：NumPy/Pure Python（遗传编程不需GPU）
- **融合层**：Sklearn Ridge/XGBoost（简单高效）

---

---

## 🎯 5. 关键洞察与总结

### 双塔模型的核心价值

1. **神经网络塔**：
   - 💪 强大的非线性拟合能力
   - ⚡ 快速训练（梯度下降）
   - ⚠️ 黑盒模型，难以解释

2. **符号回归塔**：
   - 🔍 完全可解释的数学公式
   - 🛡️ 泛化能力强，不易过拟合
   - ⏰ 训练慢（遗传编程）

3. **双塔融合**：
   - 🏆 取长补短，性能最优
   - ⚖️ 准确性 + 可解释性
   - 🚀 适合实战应用

### 选择建议

- **快速原型**：神经网络塔
- **监管要求**：符号回归塔
- **追求极致**：Stacking融合
- **生产环境**：IC动态权重融合

---

**🎓 恭喜完成双塔模型学习！**